In [1]:
def plot_geometry(x,z,ax):
    Lh = H/np.tan(beta)
    ax.plot([-H,0,Lh,max(xB(x,z)+H/10,Lh+xSV[1]+0.5)],[0,0,H,H],color='black',lw=6)
    ax.fill_between([-H,0,Lh,max(xB(x,z)+H/10,Lh+xSV[1]+0.5)],[0,0,H,H], -H/3, color='gray', alpha=0.5, label=r'Gray Fill')
    if (p_load > 0):
        ax.plot([Lh+xSV[0],Lh+xSV[0],Lh+xSV[1],Lh+xSV[1]],[H,H*(1+.2*p_load/30),H*(1+.2*p_load/30),H],color='green',lw=6)
        ax.fill_between([Lh+xSV[0],Lh+xSV[0],Lh+xSV[1],Lh+xSV[1]],[H,H*(1+.2*p_load/30),H*(1+.2*p_load/30),H], H, color='green', alpha=0.5, label=r'Gray Fill')
        ax.annotate(r'%.1f kPa' %(p_load),xy=(Lh+xSV[0],H*(1+.2*p_load/30+0.1)),color='green');
    ax.set_xlabel(r'$x$ / m')
    ax.set_ylabel(r'$z$ / m')
    ax.set(adjustable='box', aspect='equal')
    return None

In [2]:
Radius = lambda x, z: np.sqrt(x**2 + z**2)
xB = lambda x, z: x+np.sqrt(Radius(x,z)**2 - (z-H)**2)

In [3]:
#circles must not intersect slope itself
def is_valid_circle(x,z):
    return (xB(x,z) >= H/np.tan(beta))

In [4]:
def get_alpha_r(x, z):
    MFl = np.array([-x,-z])
    FBl = np.array([xB(x,z),H])
    MBl = FBl + MFl
    return np.arccos(np.dot(MFl,MBl)/(np.linalg.norm(MFl)*np.linalg.norm(MBl))) / 2

In [5]:
def plot_circle(x,z,ax):
    r = Radius(x,z)
    alpha_r = get_alpha_r(x, z)
    alpha_0 = np.arctan(-x/z)
    angle = patches.Arc((x, z), width=2*r, height=2*r, theta1=0,theta2=2*np.rad2deg(alpha_r), 
                        angle = -90+np.rad2deg(alpha_0), ls='-',fill=False,lw=2)
    ax.add_patch(angle)
    return None

In [6]:
def run_analysis(save=False):
    fig, ax = plt.subplots(figsize=(12,12))
    x_max = 0
    z_max = 0
    mu_max = 0
    e_max = 0
    etas = np.zeros((len(Mz),len(Mx)))
    for j,x in enumerate(Mx):
        for i,z in enumerate(Mz):
            if (not is_valid_circle(x,z)):
                etas[i,j] = None
            else:
                ax.plot(x,z,marker='+',markersize=10,color='black',ls='',alpha=0.5)
                #Geometrie
                r = Radius(x,z)
                MF = np.array([-x,-z])
                FB = np.array([xB(x,z),H])
                MB = FB + MF
                FS = np.array([H/np.tan(beta),H])
                MS = FS + MF
                MP = lambda psi: np.array([r * np.sin(psi),-r*np.cos(psi)])#P liegt auf Gleitkreis
                SP = lambda psi: MP(psi) - MS
                MC = lambda psi: MS + 2./3. * SP(psi) #C ist Lamellenschwerpunkt
                int_angle = lambda a, b: np.arccos(np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)))
                alpha_r = get_alpha_r(x,z)

                #Flächenintegral
                alpha_0 = np.arctan(-x/z)
                alpha_1 = alpha_0 + 2.*alpha_r
                alphas = np.linspace(alpha_0,alpha_1,100)
                A = 0
                eA = 0
                for ii in range(1,len(alphas)):
                    r_0 = np.linalg.norm(SP(alphas[ii-1]))
                    #ax.plot(FS[0]+SP(alphas[ii-1])[0],FS[1]+SP(alphas[ii-1])[1],marker='d')
                    r_1 = np.linalg.norm(SP(alphas[ii]))
                    dpsi = int_angle(SP(alphas[ii-1]),SP(alphas[ii]))
                    dA = (r_0+r_1)**2/8 * dpsi
                    A += dA
                    e = MC((alphas[ii]+alphas[ii-1])/2)[0]
                    #ax.plot(-MF[0]+MC((alphas[ii]+alphas[ii-1])/2)[0],-MF[1]+MC((alphas[ii]+alphas[ii-1])/2)[1],marker='d')
                    eA += dA*e
                e = eA/A
                F = gamma * A
                MT = F * e

                #Auflast
                if (xSV[0]+H/np.tan(beta) < xB(x,z)):
                    xSB = xB(x,z) - H/np.tan(beta)
                    load_length = np.minimum(xSB,xSV[1]) - xSV[0]
                    V = p_load * load_length
                    F += V
                    MT += (MS[0]+xSV[0] + load_length/2) * V

                #Kohäsiver Anteil
                l_FB = np.linalg.norm(FB)
                rc = alpha_r / np.sin(alpha_r) * r
                beta_F = np.arctan(FB[1]/FB[0])
                Fc = np.array([c*l_FB*np.cos(beta_F),c*l_FB*np.sin(beta_F)])
                MFc = c*l_FB * rc

                #Reibungsanteil
                Q = -np.array([0,-F]) - Fc
                alpha_r = get_alpha_r(x,z)
                xi = (1+alpha_r/np.sin(alpha_r))/2

                mu = MT / (MFc + np.linalg.norm(Q) * xi * Radius(x,z)*np.sin(phi))
                eta = 1/mu
                etas[i,j] = eta
                if (mu > mu_max):
                    mu_max = mu
                    x_max = x
                    z_max = z

    plot_geometry(x_max,z_max, ax)
    plot_circle(x_max,z_max,ax)
    X, Y = np.meshgrid(Mx, Mz)
    cp = plt.contour(X, Y, etas,levels=20)
    ax.clabel(cp, inline=True, 
              fontsize=12)
    ax.annotate(r'%.2f' %(1/mu_max),xy=(x_max,z_max),color='red');
    ax.plot(x_max,z_max,marker='+',markersize=10,color='red',ls='')
    ax.plot([0,x_max,xB(x_max,z_max)],[0,z_max,H],color='red',lw=1)
    fig.suptitle('DIN4084 old: $\mu = %.2f$, $\eta = %.2f$, $r = %.1f \,$m' %(mu_max,1/mu_max, Radius(x_max,z_max)))
    
    ax_max = np.maximum(np.max(Mz),xB(x_max,z_max)+H/10)
    ax.set_xlim(left=-H,right=ax_max)
    ax.set_ylim(top=ax_max)
    fig.tight_layout();
    if save:
        fig.savefig('bishop.pdf')
    return None

In [ ]:
def run_analysis_new(save=False):
    fig, ax = plt.subplots(figsize=(12,12))
    x_max = 0
    z_max = 0
    mu_max = 0
    e_max = 0
    etas = np.zeros((len(Mz),len(Mx)))
    for j,x in enumerate(Mx):
        for i,z in enumerate(Mz):
            if (not is_valid_circle(x,z)):
                etas[i,j] = None
            else:
                ax.plot(x,z,marker='+',markersize=10,color='black',ls='',alpha=0.5)
                #Geometrie
                r = Radius(x,z)
                MF = np.array([-x,-z])
                FB = np.array([xB(x,z),H])
                MB = FB + MF
                FS = np.array([H/np.tan(beta),H])
                MS = FS + MF
                MP = lambda psi: np.array([r * np.sin(psi),-r*np.cos(psi)])#P liegt auf Gleitkreis
                SP = lambda psi: MP(psi) - MS
                MC = lambda psi: MS + 2./3. * SP(psi) #C ist Lamellenschwerpunkt
                int_angle = lambda a, b: np.arccos(np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)))
                alpha_r = get_alpha_r(x,z)

                #Flächenintegral
                alpha_0 = np.arctan(-x/z)
                alpha_1 = alpha_0 + 2.*alpha_r
                alphas = np.linspace(alpha_0,alpha_1,100)
                A = 0
                eA = 0
                for ii in range(1,len(alphas)):
                    r_0 = np.linalg.norm(SP(alphas[ii-1]))
                    #ax.plot(FS[0]+SP(alphas[ii-1])[0],FS[1]+SP(alphas[ii-1])[1],marker='d')
                    r_1 = np.linalg.norm(SP(alphas[ii]))
                    dpsi = int_angle(SP(alphas[ii-1]),SP(alphas[ii]))
                    dA = (r_0+r_1)**2/8 * dpsi
                    A += dA
                    e = MC((alphas[ii]+alphas[ii-1])/2)[0]
                    #ax.plot(-MF[0]+MC((alphas[ii]+alphas[ii-1])/2)[0],-MF[1]+MC((alphas[ii]+alphas[ii-1])/2)[1],marker='d')
                    eA += dA*e
                e = eA/A
                F = gamma * A
                MT = F * e

                #Auflast
                if (xSV[0]+H/np.tan(beta) < xB(x,z)):
                    xSB = xB(x,z) - H/np.tan(beta)
                    load_length = np.minimum(xSB,xSV[1]) - xSV[0]
                    V = p_load * load_length
                    F += V
                    MT += (MS[0]+xSV[0] + load_length/2) * V

                #Kohäsiver Anteil
                l_FB = np.linalg.norm(FB)
                rc = alpha_r / np.sin(alpha_r) * r
                beta_F = np.arctan(FB[1]/FB[0])
                Fc = np.array([c*l_FB*np.cos(beta_F),c*l_FB*np.sin(beta_F)])
                MFc = c*l_FB * rc

                #Reibungsanteil alte Norm
                Q = -np.array([0,-F]) - Fc
                alpha_r = get_alpha_r(x,z)
                xi = (1+alpha_r/np.sin(alpha_r))/2

                #Startwert: alte Norm
                mu_0 = MT / (MFc + np.linalg.norm(Q) * xi * Radius(x,z)*np.sin(phi))
                
                #mu_ip1 = lambda mu_i: (MT - np.linalg.norm(-np.array([0,-F]) - mu_i * Fc) * xi * Radius(x,z)*np.sin(np.arctan(mu_i * np.tan(phi))))/MFc
                #mu_ip1 = lambda mu_i: (mu_i * MT)/(MFc * mu_i + np.linalg.norm(-np.array([0,-F]) - mu_i * Fc) * xi * Radius(x,z)*np.sin(np.arctan(mu_i * np.tan(phi))))
                
                #print(np.abs(mu_ip1(mu_0+1e-8)-mu_ip1(mu_0-1e-8))/2e-8)
                
                #f_test = lambda mu_i: (MT - np.linalg.norm(-np.array([0,-F]) - mu_i * Fc) * xi * Radius(x,z)*np.sin(np.arctan(mu_i * np.tan(phi))))/(MFc * mu_i) - 1
                f_test = lambda mu_i: (MT)/(MFc * mu_i + np.linalg.norm(-np.array([0,-F]) - mu_i * Fc) * xi * Radius(x,z)*np.sin(np.arctan(mu_i * np.tan(phi)))) - 1
                
                
                #counter = 0
                #while (np.abs(mu_0 - mu_ip1(mu_0)) > 0.01):
                #    counter = counter + 1
                #    if (counter > 2):
                #        #print("Required more than 10 iterations to converge")
                #        break
                #    mu_0 = mu_ip1(mu_0)
                 
                mu_1 = fsolve(f_test,mu_0)
                
                eta = 1/mu_1
                etas[i,j] = eta
                if (mu_1 > mu_max):
                    mu_max = mu_1
                    x_max = x
                    z_max = z

    plot_geometry(x_max,z_max, ax)
    #plot load
    #ax.plot([-H,0,Lh,xB(x,z)+H/10],[0,0,H,H],color='black',lw=6)
    plot_circle(x_max,z_max,ax)
    X, Y = np.meshgrid(Mx, Mz)
    cp = plt.contour(X, Y, etas,levels=20)
    ax.clabel(cp, inline=True, 
              fontsize=12)
    ax.annotate(r'%.2f' %(1/mu_max),xy=(x_max,z_max),color='red');
    ax.plot(x_max,z_max,marker='+',markersize=10,color='red',ls='')
    ax.plot([0,x_max,xB(x_max,z_max)],[0,z_max,H],color='red',lw=1)
    fig.suptitle('DIN4084 2021: $\mu = %.2f$, $\eta = %.2f$, $r = %.1f \,$m' %(mu_max,1/mu_max, Radius(x_max,z_max)))
    
    ax_max = np.maximum(np.max(Mz),xB(x_max,z_max)+H/10)
    ax.set_xlim(left=-H,right=ax_max)
    ax.set_ylim(top=ax_max)
    fig.tight_layout();
    if save:
        fig.savefig('bishop.pdf')
    return None